In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os


def detect_and_crop_handwriting(image):
    # image = cv2.imread(image)

    if image is None:
        print(f"Failed to load image at {image}")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    height, width = image.shape[:2]
    center_x = width // 2
    center_y = height // 2
    max_offset = -1
    max_offset_contour = None

    for contour in contours:
        M = cv2.moments(contour)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])

        offset = np.sqrt((center_x - cX) ** 2 + (center_y - cY) ** 2)

        if offset > max_offset:
            max_offset = offset
            max_offset_contour = contour

    if max_offset_contour is not None:
        x, y, w, h = cv2.boundingRect(max_offset_contour)

        aspect_ratio = float(w) / h

        if aspect_ratio > 1:
            y_padding = int((w - h) / 2)
            x_padding = 0
        else:
            x_padding = int((h - w) / 2)
            y_padding = 0

        x -= x_padding
        w += 2 * x_padding
        y -= y_padding
        h += 2 * y_padding

        x = max(x, 0)
        w = min(w, width)
        y = max(y, 0)
        h = min(h, height)

        cropped_image = image[y:y + h, x:x + w]

        # resized_image = cv2.resize(cropped_image, (28, 28))
        resized_image = cv2.resize(cropped_image, (300, 300))
        # resized_gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

        # resized_gray = resized_gray.reshape((28, 28, 1))

        # return resized_gray
        return resized_image

    else:
        print('No handwriting detected in the image.')
        return None


data_path = 'data_test'
size = 28
folders = ['0','1','2','3','4','5','6','7','8','9']
X, y = [], []
kernel = np.ones((5, 5), np.uint8)
for folder in folders:
    folder_path = os.path.join(data_path, folder)
    images = os.listdir(folder_path)
    for image_name in images:
        image_path = os.path.join(folder_path, image_name).replace("\\","/")
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        # image = detect_and_crop_handwriting(image)
        # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        print(image_path)
        image = detect_and_crop_handwriting(image)
        image = cv2.dilate(image, kernel, iterations=1)
        image = cv2.resize(image, (size, size))  # Resize the image to 28x28 pixels
        X.append(image.flatten())  # Flatten the image and add it to the feature matrix
        y.append(int(folder))  # Add the corresponding label

X_data = np.array(X)
y_data = np.array(y)


data_test/0/0_20230603230116.png


error: OpenCV(4.7.0) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0x981fb336::Set<1,-1,-1>,struct cv::impl::A0x981fb336::Set<0,2,5>,2>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1
